In [339]:
!pip install pulp
import numpy as np
import pandas as pd
from IPython.display import clear_output #for UI
from pulp import * #for math optimization

In [329]:
#reading the csv file and dropping vegNonVeg column as it is not needed
sampleData = pd.read_csv('/content/drive/MyDrive/nutritionalValues.csv').drop(columns = 'VegNovVeg', axis=1)

In [330]:
#for the optimization algortihms, we only need some columns
optcols = ['Food_items', 'Calories', 'Fats', 'Proteins','Carbohydrates']
opt = sampleData[optcols]
#converting fats, proteins and carbs columns to float-type
flt = ['Fats', 'Proteins','Carbohydrates']
opt[flt].astype(float)


,Fats,Proteins,Carbohydrates
0,0.2,2.4,4.1
1,15.0,2.0,8.5
2,0.3,1.1,23.0
3,1.5,10.0,49.0
4,0.4,14.0,77.0
...,...,...,...
84,11.0,3.5,24.0
85,8.4,3.2,28.0
86,0.2,1.8,81.0
87,30.0,7.7,59.0


In [331]:
#to make sure any dish is not being repeated on different days
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']

#now, we split the data randomly into 8 equal parts
splitValuesDays = np.linspace(0,len(opt),8).astype(int)

#we divided the data into 8 equal parts, because the dataset may/may not be divisible by 7
#so the following will make sure any data point is not missed
splitValuesDays[-1] = splitValuesDays[-1] - 1

#Now we randomise the food into the days, lets define a function
def randomisedDays():
   fracValues = opt.sample(frac=1).reset_index().drop('index',axis=1)
   dayData = [] #an empty list to be appended with the data of a specific day
   for z in range(len(splitValuesDays)-1):
    dayData.append(fracValues.loc[splitValuesDays[z]:splitValuesDays[z+1]]) #this slices splitValuesDays, and loc is a function to access rows
   return dict(zip(days, dayData))

In [332]:
#this is the same implementation as randomisedDays() but here, we are randomising meals
meals = ['Breakfast', 'Snack 1', 'Lunch', 'Snack 2', 'Dinner']
splitValuesMeal = np.linspace(0,splitValuesDays[1],len(meals)+1).astype(int)
splitValuesMeal[-1] = splitValuesMeal[-1] - 1

import random
def randomisedMeals(dayData):
    mealsData = {}
    for meal in meals:
        fracValues = dayData.sample(frac=1).reset_index(drop=True)
        mealData = fracValues.sample(frac=random.uniform(0.5, 1.0)).reset_index(drop=True)
        mealsData[meal] = mealData
    return mealsData


**Defining calorie intake by dividing it equally into fats, carbs and proteins**

Using the data provided by the World Health Organization (WHO), the Academy of Nutrition and Dietetics, and the American Heart Association, we can make a nutrition plan based on the following data:

1. 1 gram of protein per weight (in kg) of the person
2. carbohydrates should be half of the calories you're aiming to take
3. the remaining calories should be fats


In [333]:
def nutritional_calories(weight, cals):
  pr_cals = weight*4 #it is assumed that 1 gram of protein gives approx 4 calories
  carb_cals = cals/2
  fat_cals = cals - (pr_cals + carb_cals)
  finalcals = {"Protein calories" : pr_cals , "Carbohydrate calories" : carb_cals, "Fat calories" : fat_cals}
  return finalcals

In [334]:
#converting the nutritional values to grams

def nutritional_grams(calsdict):
  pr_grams = calsdict['Protein calories']/4 #approx 4 calories per gram of proteins
  carb_grams = calsdict['Carbohydrate calories']/4 #approx 4 calories per gram of carbohydrates
  fat_grams = calsdict['Fat calories']/9 #approx 9 calories per gram of fats
  finalgrams = {"Protein grams" : pr_grams, "Carbohydrate grams" : carb_grams , "Fat grams" : fat_grams}
  return finalgrams

**Optimization**

The problem we have is a continuous linear optimization, which can be solved through multiple algorithms (barrier, ellipsoid, cutting plane methods etc) but the one we will be using is [Simplex method](https://optimization.cbe.cornell.edu/index.php?title=Simplex_algorithm).
It is the most well-known algorithm for solving linear programming problems (LPP). It's main motive is to find the extreme points in the feasible region; which is achieved through iteratively moving of the algorithm along the edges of the feasible region until an extreme point is found.

**Function and constraints**

The simplex method works on minimizing/maximising a function, provided we comply with the given constraints (these contraints are basically how feasible region is formed), and then apply the algorithm.

In this problem, we are minimizing the calorie intake, while trying to comply with the carbohydrate, fat and protein recommended intake. Hence, this gives rise to a linear problem and as the function is to be minimized, we use the [cost-loss function](https://www.enjoyalgorithms.com/blog/loss-and-cost-functions-in-machine-learning).

The function being used is:
# *L(x) =   Cᵀ . X   =   Σⁿᵢ₌₁ [cᵢ  xᵢ]*
>* Cᵀ is the transpose of the vector c, representing the row vector of costs or coefficients associated with each variable or feature
* X is the vector of variables or features
* [cᵢ  xᵢ] is representing the cost associated with the iᵗʰ variable



---
Now, we will explain the constraints; which in this case are the preferred amount of proteins, fats and carbohydrates we need. For this we will first define 3 different vectors (matrices) for each feature respectively:

1. Proteins: p = [ p₁, p₂, ... , pₙ ]
2. Fats: f = [ f₁, f₂, ... , fₙ ]
3. Carbohydrates: b = [ b₁, b₂, ... ,bₙ ] - not using C as it is used in loss function

Now for the constraints,

# { Σⁿᵢ₌₁ [pᵢ  xᵢ] ≥ P
# { Σⁿᵢ₌₁ [fᵢ  xᵢ] ≥ F
# { Σⁿᵢ₌₁ [bᵢ  xᵢ] ≥ B

where P, F and B are the amount of proteins, fats and carbohydrates we need to consume.


---

now that we have the function to minimize and the constraints, we can easily implement this in code. The library that will be used is [PuLP](https://coin-or.github.io/pulp/) which is a widely used Linear Programming modeler (used in resource allocation, network optimization, scheduling etc.)




---






Also, the optimization does not divide the meals equally and sufficiently. The model will give you foods in random quantities and it is hard to eat meals that you done like at the wrong time.

For this, the guidelines to eat (in calories) at specific times are:

* Around 10% for the snack number 1
* Around 10% for the snack number 2
* Around 30% for dinner
* 35% for lunch
* 15 % for breakfast

*generally given by nutritionists, dieticians, and health organizations*

To achieve this, we will randomly split our items into 5 (per day) and the diversity will be:
1. d(breakfast)=1/0.15
2. d(lunch)=1/0.35
3. d(dinner)=1/0.30
4. d(snacks)=1/0.01

Impelementing all this in the following code:

In [335]:
daysData = randomisedDays()
def dietModel(constr, day, weight, cals, food, data):
  gms = nutritional_grams(nutritional_calories(weight, cals)) #dictionary for grams required for intake
  P = gms['Protein grams']
  F = gms['Fat grams']
  B = gms['Carbohydrate grams']
  dayData = daysData[day]
  dayData = dayData[dayData.Calories !=0] #removes any rows that have calories=0

  meal = dayData.Food_items.tolist() #converts the food_items column to list
  c = dayData.Calories.tolist() #converts the calories column to list
  np.random.shuffle(meal)

  x  = pulp.LpVariable.dicts( "x", indices = meal, lowBound=0, upBound=1.5, cat='Continuous', indexStart=[] )

  p = dayData.Proteins.tolist()
  f = dayData.Fats.tolist()
  b = dayData.Carbohydrates.tolist()

  #using PuLP to optimize
  constr = pulp.LpProblem("DietCalculator", LpMinimize) #creates a new Linear Programming Problem (LPP)
  #objective function:
  constr += pulp.lpSum([x[meal[i]]*c[i] for i in range(len(meal))]) #lpSum adds everything (Σ) and for loop iterates over each food_item
  #constraints:
  splitMeal = {'Snack 1': 0.10, 'Snack 2':0.10,'Breakfast': 0.15,'Lunch':0.35, 'Dinner':0.30}
  divMeal = splitMeal[food]
  constr += pulp.lpSum([x[meal[i]]*p[i] for i in range(len(x))]) >= P*divMeal #constraint for proteins
  constr += pulp.lpSum([x[meal[i]]*f[i] for i in range(len(x))]) >= F*divMeal  #constraint for fats
  constr += pulp.lpSum([x[meal[i]]*b[i] for i in range(len(x))]) >= B*divMeal #constraint for carbohydrates
  #solving the problem using pulp's solving function
  constr.solve()

  #formatting the solution into a dataframe:
  vars, values = [] , []
  for l in constr.variables():
    var = l.name #name of decision variable
    val = l.varValue
    vars.append(var)
    values.append(val)
  values = np.array(values).round(2).astype(float) #array for decision variables, upto 2 decimal places with type float
  #solution into df
  solution = pd.DataFrame(np.array([meal, values]).T, columns = ['Food Items', 'Quantity']) #making df using pandas
  solution['Quantity'] = solution.Quantity.astype(float)
  solution = solution[solution['Quantity'] != 0] #eliminating any items with quantity=0
  solution.Quantity *= 100 #converting into grams
  solution = solution.rename(columns = {'Quantity' : 'Quantity (grams)'})
  return solution

#For the final model implementation
def baseModel(weight, cals):
  result = []
  for day in days:
    constr = pulp.LpProblem("DietCalculator", LpMinimize)
    print(f'Building a model for {day}')
    result.append(dietModel(constr, day, weight, cals))
  return dict(zip(days, result))



In [336]:
#final model that runs dietModel and baseModel respectively
def finalModel(weight, cals):
    res_model = []
    for day in days:
        dayData = daysData[day]
        mealsData = randomisedMeals(dayData)
        mealModel = {}
        for meal in meals:
            constr = pulp.LpProblem("DietCalculator", LpMinimize)
            solModel = dietModel(constr, day, weight, cals, meal, mealsData)

            mealModel[meal] = solModel
        res_model.append(mealModel)
    return dict(zip(days,res_model))


In [337]:
#execution function
def execModel(diet, selected_days):
    for day in selected_days:
        print(f"\033[1m{day}\033[0m")  # Bold day
        print("\n")  # Add two new lines after printing the day
        meal_labels = []
        for meal in meals:
            meal_data = diet[day][meal]
            if not meal_data.empty:
                # Add a label for the meal if it hasn't been added yet
                if meal not in meal_labels:
                    print(f"\033[4m\033[3m{meal}\033[0m")  # Underline and italicize meal name
                    meal_labels.append(meal)
                # Format the meal data
                meal_data_formatted = meal_data[['Food Items', 'Quantity (grams)']].copy()
                meal_data_formatted['Food Items'] = meal_data_formatted['Food Items'].apply(lambda x: x.ljust(25))
                # Print the meal data
                print(meal_data_formatted.to_string(index=False, header=False))
                print("---------------------------------")
        print("________________________________________________________________________________________________________________")


In [342]:
#final execution and UI: loading bar, ascii art
import time

def zephyr_art():
    print("""
                 _                       _ _      _             _
  _____   _ _ __ | |__   ___ _ __      __| (_) ___| |_     _ __ | | __ _ _ __  _ __   ___ _ __
 |_  / | | | '_ \| '_ \ / _ \ '__|    / _` | |/ _ \ __|   | '_ \| |/ _` | '_ \| '_ \ / _ \ '__|
  / /| |_| | |_) \| | | |  __/ |      | (_| | |  __/ |_    | |_) | | (_| | | | | | | |  __/ |
 /___|\__, | .__/|_| |_|\___|_|       \__,_|_|\___|\__|   | .__/|_|\__,_|_| |_|_| |_|\___|_|
      |___/|_|                                            |_|
""")

def loading_bar(length):
    for _ in range(length):
        print("-", end="", flush=True)
        time.sleep(0.1)

def main():
    clear_output(wait=True)
    zephyr_art()
    loading_bar(92)
    clear_output(wait=True)

    print("Enter weight (in kgs):")
    weight = float(input())
    print("Enter target calories:")
    cals = float(input())
    print("Select the days you want to view (separate with commas, or type 'all' for all days):")
    print("Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday")
    days_input = input("Your choice: ").strip().lower()
    if days_input == "all":
        selected_days = days
    else:
        selected_days = [day.strip().capitalize() for day in days_input.split(",")]

    # Get the diet plan for the selected days
    diet = finalModel(weight, cals)


    execModel(diet, selected_days)

if __name__ == "__main__":
    main()


Enter weight (in kgs):
45.6
Enter target calories:
2100
Select the days you want to view (separate with commas, or type 'all' for all days):
Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday
Your choice: monday, tuesday
Monday


Breakfast
Dal Fry                   150.0
Rohu Curry                 54.0
Sitafal                   105.0
---------------------------------
Snack 1
Sitafal                   150.0
Bananas                    60.0
Dahi                       23.0
---------------------------------
Lunch
Bananas                   150.0
Kadhi                      38.0
Dal Fry                   150.0
Surmai                    150.0
Onion Pakoda              150.0
Vanilla Ice cream         148.0
Dahi                       36.0
---------------------------------
Snack 2
Idli                       60.0
Chicken Tandoori          150.0
Rohu Curry                 23.0
---------------------------------
Dinner
Chicken Tandoori          150.0
Rohu Curry                150.0
Strawbe